# Validate Motion State

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import fetch_data
import numpy as np
import correct_drift
import datetime
from scipy import signal
from scipy.spatial.transform import Rotation as R
from ahrs.filters import Madgwick
from ahrs.filters import Mahony

import importlib
importlib.reload(fetch_data)

<module 'fetch_data' from '/home/tyler/Documents/Matt3r/IMU_validation/fetch_data.py'>

Select Data

In [29]:
# Hamid data
start_date_str = '2023-06-28'
end_date_str = '2023-06-28'
organization_id = 'hamid'
can_k3y_id = '17700cf8'
imu_k3y_id = '17700cf8'

In [22]:
# Amir M data
start_date_str = '2023-06-28'
end_date_str = '2023-06-28'
organization_id = 'amirm'
can_k3y_id = 'c9a6b512'
imu_k3y_id = 'c9a6b512'

Get the CAN Server Data

In [30]:
event_dict = fetch_data.get_events(can_k3y_id, organization_id, start_date_str, end_date_str)
can_df = fetch_data.get_can_data(can_k3y_id, organization_id, start_date_str, end_date_str)

Get the IMU Data

In [31]:
# use if we have clock data
imu_df = fetch_data.get_imu_data(imu_k3y_id, organization_id, start_date_str, end_date_str)

Correct the Rotation Matrix for Hamid's Data

In [33]:
GLOVE_TO_CONSOLE_MTX = np.array([[0, 0, 1], [0.0856, 0.9963, 0], [-0.9963, 0.0856, 0]])
S3_K3Y_VEH_ROT_MTX_CONSOLE = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
S3_K3Y_VEH_ROT_MTX = np.matmul(S3_K3Y_VEH_ROT_MTX_CONSOLE, GLOVE_TO_CONSOLE_MTX)
CORRECTION_MATRIX = S3_K3Y_VEH_ROT_MTX_CONSOLE@np.linalg.inv(S3_K3Y_VEH_ROT_MTX)

# az = 9.83
# # ay = 0.835
# ay = 0.987
# g = np.sqrt(az**2 + ay**2)
# TILT_ROT = np.array([[1, 0, 0], [0, az/g, -ay/g], [0, ay/g, az/g]])

# CORRECTION_MATRIX = S3_K3Y_VEH_ROT_MTX_CONSOLE@TILT_ROT@np.linalg.inv(S3_K3Y_VEH_ROT_MTX)

if organization_id == 'hamid':
    acc_np = imu_df[['correct_timestamp', 'lr_acc(m/s^2)', 'bf_acc(m/s^2)', 'vert_acc(m/s^2)']].to_numpy()
    vehicle_ac = np.matmul(CORRECTION_MATRIX, acc_np[:, 1:].T)
    vehicle_ac_np = np.column_stack((acc_np[:,0], vehicle_ac.T))
    imu_df = pd.DataFrame(vehicle_ac_np, columns=['correct_timestamp', 'lr_acc(m/s^2)', 'bf_acc(m/s^2)', 'vert_acc(m/s^2)'])

Use the speed data to create stationary and driving states

In [34]:
stationary_speed = 0.5
drive_duration = 1

speed_df = can_df[can_df['speed'].notna()].copy()
speed_df.reset_index(drop=True, inplace=True)
speed_df['driving'] = abs(speed_df['speed']) > stationary_speed
dr_start_times = speed_df[speed_df['driving'].astype(int).diff() == 1]['timestamp'].to_list()
dr_end_times = speed_df[speed_df['driving'].astype(int).diff() == -1]['timestamp'].to_list()

# get the driving states
dr_df_states = []
for i in range(min(len(dr_start_times),len(dr_end_times))):
    if dr_end_times[i] - dr_start_times[i] > drive_duration:
        dr_df_states.append(imu_df[(imu_df['correct_timestamp'] >= dr_start_times[i]) 
                        & (imu_df['correct_timestamp'] <= dr_end_times[i])])
dr_df = pd.concat(dr_df_states, ignore_index=True)

In [38]:
# get the driving state set based on the stationary states

dr_df = imu_df
for state_num in range(len(event_dict['stationary_state'])):
    ss_start_time = event_dict['stationary_state'][state_num]['start']
    ss_end_time = event_dict['stationary_state'][state_num]['end']
    dr_df = dr_df[(dr_df['correct_timestamp'] <= ss_start_time) | (dr_df['correct_timestamp'] >= ss_end_time)]
dr_df_list = []
for state in event_dict['driving_state']:
    dr_start_time = state['start']
    dr_end_time = state['end']
    dr_df_list.append(dr_df[(dr_df['correct_timestamp'] >= dr_start_time) & (dr_df['correct_timestamp'] <= dr_end_time)])
dr_df = pd.concat(dr_df_list, ignore_index=True)

Use a cutoff acceleration to determine motion and compare with speed data

In [64]:
acc_cutoff = 1.2

imu_df['norm_acc'] = np.sqrt(imu_df['lr_acc(m/s^2)']**2 + imu_df['bf_acc(m/s^2)']**2 + imu_df['vert_acc(m/s^2)']**2)
dr_proxy_df = imu_df[imu_df['norm_acc'] > acc_cutoff]

proxy_set = set(dr_proxy_df['correct_timestamp'].to_list())
truth_set = set(dr_df['correct_timestamp'].to_list())

# should be close to 1.00
stationary_validation = len(truth_set.intersection(proxy_set)) / len(proxy_set)
print(stationary_validation)

0.7479260369815093


In [204]:
similarity = len(truth_set.intersection(proxy_set)) / len(truth_set.union(proxy_set))
similarity

0.7500497760079642

Optimize the Acceleration Parameter

In [197]:
print(dr_proxy_df['correct_timestamp'].diff().max())

44875.84837460518


Filter Out Warmup Time

In [6]:
warmup_start = 1687886372
warmup_end = 1687886435
imu_df = imu_df[(imu_df['correct_timestamp'] <= warmup_start) | (imu_df['correct_timestamp'] >= warmup_end)]

Get Stationary State Data Metrics

In [17]:
# stitching together all stationary state data
imu_ss_states = []
can_ss_states = []
for state_num in range(len(event_dict['stationary_state'])):
    ss_start_time = event_dict['stationary_state'][state_num]['start']
    ss_end_time = event_dict['stationary_state'][state_num]['end']
    imu_ss_states.append(imu_df[(imu_df['correct_timestamp'] >= ss_start_time) 
                       & (imu_df['correct_timestamp'] <= ss_end_time)])
    can_ss_states.append(can_df[(can_df['timestamp'] >= ss_start_time) 
                        & (can_df['timestamp'] <= ss_end_time)])
imu_ss_df = pd.concat(imu_ss_states, ignore_index=True)
can_ss_df = pd.concat(can_ss_states, ignore_index=True)

print('left/right acceleration:')
print(f'imu mean: {imu_ss_df["lr_acc(m/s^2)"].mean()}')
print(f'imu max: {imu_ss_df["lr_acc(m/s^2)"].max()}')
print(f'imu min: {imu_ss_df["lr_acc(m/s^2)"].min()}')
print(f'can mean: {can_ss_df["lr_acc"].mean()}')
print(f'can max: {can_ss_df["lr_acc"].max()}')
print(f'can min: {can_ss_df["lr_acc"].min()}')
print('')
print('back/front acceleration:')
print(f'imu mean: {imu_ss_df["bf_acc(m/s^2)"].mean()}')
print(f'imu max: {imu_ss_df["bf_acc(m/s^2)"].max()}')
print(f'imu min: {imu_ss_df["bf_acc(m/s^2)"].min()}')
print(f'can mean: {can_ss_df["bf_acc"].mean()}')
print(f'can max: {can_ss_df["bf_acc"].max()}')
print(f'can min: {can_ss_df["bf_acc"].min()}')
print('')
print('vertical acceleration:')
print(f'imu mean: {imu_ss_df["vert_acc(m/s^2)"].mean()}')
print(f'imu max: {imu_ss_df["vert_acc(m/s^2)"].max()}')
print(f'imu min: {imu_ss_df["vert_acc(m/s^2)"].min()}')
print(f'can mean: {can_ss_df["vert_acc"].mean()}')
print(f'can max: {can_ss_df["vert_acc"].max()}')
print(f'can min: {can_ss_df["vert_acc"].min()}')
print('')
print('normalized acceleration:')
imu_norm = np.sqrt(imu_ss_df['lr_acc(m/s^2)']**2 + imu_ss_df['bf_acc(m/s^2)']**2 + imu_ss_df['vert_acc(m/s^2)']**2)
can_norm = np.sqrt(can_ss_df['lr_acc']**2 + can_ss_df['bf_acc']**2 + can_ss_df['vert_acc']**2)
print(f'imu mean: {imu_norm.mean()}')
print(f'imu max: {imu_norm.max()}')
print(f'imu min: {imu_norm.min()}')
print(f'can mean: {can_norm.mean()}')
print(f'can max: {can_norm.max()}')
print(f'can min: {can_norm.min()}')

ValueError: No objects to concatenate

In [105]:
# driving state metrics

# dr_df = imu_df
# for state_num in range(len(event_dict['stationary_state'])):
#     ss_start_time = event_dict['stationary_state'][state_num]['start']
#     ss_end_time = event_dict['stationary_state'][state_num]['end']
#     dr_df = dr_df[(dr_df['correct_timestamp'] <= ss_start_time) | (dr_df['correct_timestamp'] >= ss_end_time)]

print('left/right acceleration:')
print(f'imu mean: {dr_df["lr_acc(m/s^2)"].mean()}')
print(f'imu max: {dr_df["lr_acc(m/s^2)"].max()}')
print(f'imu min: {dr_df["lr_acc(m/s^2)"].min()}')
print('')
print('back/from acceleration:')
print(f'imu mean: {dr_df["bf_acc(m/s^2)"].mean()}')
print(f'imu max: {dr_df["bf_acc(m/s^2)"].max()}')
print(f'imu min: {dr_df["bf_acc(m/s^2)"].min()}')
print('')
print('vertical acceleration:')
print(f'imu mean: {dr_df["vert_acc(m/s^2)"].mean()}')
print(f'imu max: {dr_df["vert_acc(m/s^2)"].max()}')
print(f'imu min: {dr_df["vert_acc(m/s^2)"].min()}')
print('')
print('normalized acceleration:')
imu_norm = np.sqrt(dr_df['lr_acc(m/s^2)']**2 + dr_df['bf_acc(m/s^2)']**2 + dr_df['vert_acc(m/s^2)']**2)
print(f'imu mean: {imu_norm.mean()}')
print(f'imu max: {imu_norm.max()}')
print(f'imu min: {imu_norm.min()}')

left/right acceleration:
imu mean: 0.043944221051727086
imu max: 6.32931139896168
imu min: -4.045026772044387

back/from acceleration:
imu mean: 0.13444813589348042
imu max: 5.747378041296793
imu min: -4.16723471812193

vertical acceleration:
imu mean: 0.0943325911208353
imu max: 4.894458958086332
imu min: -3.6047239625559415

normalized acceleration:
imu mean: 0.8784694959040678
imu max: 6.536425891773417
imu min: 0.007551239919562279


Filter Only Driving States

In [15]:
# Check the speed of the car

df = dr_proxy_df

window = 120
count = 0

for state_num in range(len(event_dict['driving_state'])):
    dr_start_time = event_dict['driving_state'][state_num]['start']
    dr_end_time = event_dict['driving_state'][state_num]['end']

    while (dr_end_time - dr_start_time > window):
        dr_motion = df[(df['correct_timestamp'] >= dr_start_time) 
                        & (df['correct_timestamp'] <= dr_start_time + window)]
        can_motion = can_df[(can_df['timestamp'] >= dr_start_time) 
                            & (can_df['timestamp'] <= dr_start_time + window)]
        dr_start_time += window

        count += 1
        # if ((len(dr_motion) != 0) and (len(can_motion) != 0) and (count < 20)):
        if ((len(dr_motion) != 0) and (len(can_motion) != 0)):
            print(count)
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

            fig.set_size_inches(10,6)

            ax1.scatter(x=can_motion['timestamp'] - dr_start_time + window, y=-can_motion['lr_acc'], s=1, color='red', label='CANserver')
            ax1.scatter(x=dr_motion['correct_timestamp'] - dr_start_time + window, y=dr_motion['lr_acc(m/s^2)'], s=1, label='IMU')
            ax1.set_ylim(-15,15)

            ax2.scatter(x=can_motion['timestamp'] - dr_start_time + window, y=can_motion['bf_acc'], s=1, color='red', label='CANserver')
            ax2.scatter(x=dr_motion['correct_timestamp'] - dr_start_time + window, y=dr_motion['bf_acc(m/s^2)'], s=1, label='IMU')
            ax2.set_ylim(-15,15)

            ax3.scatter(x=can_motion['timestamp'] - dr_start_time + window, y=can_motion['vert_acc'], s=1, color='red', label='CANserver')
            ax3.scatter(x=dr_motion['correct_timestamp'] - dr_start_time + window, y=dr_motion['vert_acc(m/s^2)'], s=1, label='IMU')
            ax3.set_ylim(-15,15)

            fig.suptitle("IMU Acceleraion Metrics for Driving State Data")
            fig.text(0.5, 0.0, 'Elapsed Time (s)', ha='center')

            ax1.set_ylabel("Left/Right Acceleration ($m/s^2$)")
            ax2.set_ylabel("Back/Front Acceleration ($m/s^2$)")
            ax3.set_ylabel("Vertical Acceleration ($m/s^2$)")

            ax1.legend()
            ax2.legend()
            ax3.legend()
            plt.tight_layout()
            plt.show()

            fig, ax = plt.subplots()

            ax.scatter(x=can_motion['timestamp'] - dr_start_time + window, y=can_motion['speed'], s=1, color='red', label='CANserver')
            ax.set_ylim(-10,100)

            fig.suptitle("IMU Acceleraion Metrics for Driving State Data")
            fig.text(0.5, 0.0, 'Elapsed Time (s)', ha='center')

            ax.set_ylabel("Left/Right Acceleration ($m/s^2$)")

            plt.show()